In [1]:
DT_NODASH = "20220427"

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pydatafabric.vault_utils import get_secrets

oracle_info = get_secrets(mount_point="datafabric",path="oracle/datafabric/oracle-db")

In [5]:
from pydatafabric.ye import get_spark

spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/ojdbc8.jar")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/28 21:52:20 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/28 21:52:20 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/28 21:52:20 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/28 21:52:20 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [6]:
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

In [7]:
df = spark.read.format("jdbc") \
    .option("url", oracle_info['jdbc_url']) \
    .option("user", oracle_info['user']) \
    .option("password", oracle_info['password']) \
    .option("driver", "oracle.jdbc.driver.OracleDriver") \
    .option("dbtable", "(SELECT TABLESPACE_NAME FROM USER_TABLESPACES) a") \
    .load()

In [8]:
df.show()

+---------------+
|TABLESPACE_NAME|
+---------------+
|         SYSTEM|
|         SYSAUX|
|       UNDOTBS1|
|           TEMP|
|          USERS|
+---------------+



In [9]:
df = spark.read.format("jdbc") \
    .option("url", oracle_info['jdbc_url']) \
    .option("user", oracle_info['user']) \
    .option("password", oracle_info['password']) \
    .option("driver", "oracle.jdbc.driver.OracleDriver") \
    .option("dbtable", "all_tables") \
    .load()

In [10]:
df.show()

+-----+----------------+---------------+--------------+--------+------+-------------+-------------+------------+--------------+-----------------+------------------+------------+--------------------+------------+------------+---------------+-------+---------+-----------------+---------------+------------+---------+---------+--------------+-------------------------+-------------------+----------+----------+-----+----------+-----------------+-------------------+-----------+--------+---------+---------+------+-----------+-----------+----------------+------------+------------+----------+--------+------------+----------+-------------+------------+-----------+------------+-------+---------+---------------+------------+----------+-----------------+-------------+------------+--------------+--------+-----------------+-------------------+--------------------+------------------+-----------------+----------+-------+------------------+---------------------+--------+------+----------+----------------

In [11]:
from pydatafabric.gcp import df_to_bq_table, bq_insert_overwrite

df_to_bq_table(df, "temp_1d", "temp_oracle_all_tables", project="emart-datafabric") # 프로젝트 꼭 지정
bq_insert_overwrite(f"SELECT *, parse_date('%Y%m%d', '{DT_NODASH}') as dt from temp_1d.temp_oracle_all_tables", "emart-datafabric.temp_1d.oracle_all_tables", partition="dt")
    

destination: emart-datafabric.temp_1d.oracle_all_tables$20220427
total_rows: 2349
slot_secs: 1.357



In [12]:
from pydatafabric.gcp import get_bigquery_client

bq = get_bigquery_client(project="emart-datafabric") # 프로젝트 꼭 지정
r = bq.query(f"SELECT * FROM temp_1d.oracle_all_tables WHERE dt=parse_date('%Y%m%d', '{DT_NODASH}') LIMIT 10")

In [13]:
r.result().total_rows


10

In [14]:
df = r.result().to_dataframe()
df

,OWNER,TABLE_NAME,TABLESPACE_NAME,CLUSTER_NAME,IOT_NAME,STATUS,PCT_FREE,PCT_USED,INI_TRANS,MAX_TRANS,...,INMEMORY_SERVICE,INMEMORY_SERVICE_NAME,CONTAINER_MAP_OBJECT,MEMOPTIMIZE_READ,MEMOPTIMIZE_WRITE,HAS_SENSITIVE_COLUMN,ADMIT_NULL,DATA_LINK_DML_ENABLED,LOGICAL_REPLICATION,dt
0,CTXSYS,DR$INDEX_CDI_COLUMN,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
1,CTXSYS,DR$SQE,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
2,CTXSYS,DR$WAITING,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
3,CTXSYS,DR$SDATA_UPDATE,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
4,CTXSYS,DR$PENDING,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
5,CTXSYS,DR$UNINDEXED,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
6,CTXSYS,DR$ONLINE_PENDING,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
7,CTXSYS,DR$PARALLEL,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
8,CTXSYS,DR$TREE,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27
9,CTXSYS,DR$THS_FPHRASE,None,None,None,VALID,0E-9,0E-9,0E-9,0E-9,...,None,None,NO,DISABLED,DISABLED,NO,NO,NO,ENABLED,2022-04-27


In [15]:
spark.stop()